In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 1. 데이터 로딩
df = pd.read_excel("ow_player_logs_softlabel_0to1.xlsx")  # 파일 경로에 맞게 수정하세요.

# 2. 설정
POSITIONS = ["탱커", "딜러", "힐러"]
STAT_COLS = ["처치", "도움", "죽음", "피해", "치유", "경감"]

# 플레이어별 실력 점수를 저장할 딕셔너리 및 각 포지션별 모델/Scaler 저장을 위한 dict
position_scores = defaultdict(dict)
position_models = {}
position_scalers = {}  # 포지션별 scaler 재사용

# 3. 플레이어별 실력 점수를 [0, 5000] 범위로 출력하는 모델 정의
class PlayerSkillModel(nn.Module):
    def __init__(self, input_dim=6):
        super(PlayerSkillModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(16, 8),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(8, 1)
        )
    def forward(self, x):
        raw = self.net(x)
        # Sigmoid로 [0,1] 범위로 조정한 후 5000을 곱하여 [0, 5000] 범위를 얻음.
        score = torch.sigmoid(raw) * 5000  
        return score

# 4. 포지션별 학습 루프
for pos in POSITIONS:
    pos_df = df[df["포지션"] == pos].copy()
    players = pos_df["이름"].unique()
    
    inputs = []  # 개별 플레이어의 스탯
    targets = [] # 목표 실력 점수 (데이터의 soft_label * 5000)
    for _, row in pos_df.iterrows():
        stats = row[STAT_COLS].fillna(0).values.astype(np.float32)
        # soft_label이 [0,1] 범위라면, 이를 5000 배하여 목표 값을 설정
        target = float(row["soft_label"]) * 5000  
        inputs.append(stats)
        targets.append(target)
    
    # 포지션별로 StandardScaler를 학습하고 저장
    scaler = StandardScaler()
    inputs_scaled = scaler.fit_transform(inputs)
    position_scalers[pos] = scaler  # 이후 예측 및 시각화에 재사용
    
    X = torch.tensor(inputs_scaled, dtype=torch.float32)
    y = torch.tensor(targets, dtype=torch.float32).unsqueeze(1)
    
    # 모델 초기화 (포지션마다 별도 모델 사용)
    model = PlayerSkillModel(input_dim=6)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    criterion = nn.MSELoss()
    
    # 학습 루프 (예시: 300 에포크)
    for epoch in range(300):
        model.train()
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
        if epoch % 100 == 0:
            print(f"[{pos}] Epoch {epoch} | Loss: {loss.item():.4f}")
    
    # 학습 완료 후 포지션별 모델 저장
    position_models[pos] = model
    
    # 포지션 내 각 플레이어의 평균 스탯에 대해 모델 예측
    for player in players:
        player_rows = pos_df[pos_df["이름"] == player]
        if not player_rows.empty:
            avg_stats = player_rows[STAT_COLS].mean().fillna(0).values.astype(np.float32)
            # 저장된 scaler로 전처리 (재사용)
            avg_scaled = scaler.transform([avg_stats])[0]
            with torch.no_grad():
                score = model(torch.tensor(avg_scaled, dtype=torch.float32).unsqueeze(0)).item()
            position_scores[player][pos] = round(score, 2)

# 5. 결과 저장: 플레이어별 포지션 실력 점수를 엑셀 파일로 내보내기
score_df = pd.DataFrame.from_dict(position_scores, orient="index").fillna(0)
score_df.to_excel("player_position_scores_final.xlsx")
print("✅ 점수 저장 완료: player_position_scores_final.xlsx")

# 6. 학습된 모델을 포지션별로 저장
for pos, model in position_models.items():
    torch.save(model.state_dict(), f"score_model_final_{pos}.pt")

# 7. 분포 시각화 (각 포지션별 예측 점수와 실제 soft_label 기반 목표값 비교)
fig, axs = plt.subplots(1, len(POSITIONS), figsize=(18, 5))
for i, pos in enumerate(POSITIONS):
    pos_df = df[df["포지션"] == pos].copy()
    # 목표: soft_label을 5000 배해서 [0,5000] 범위로 맞춤.
    targets = pos_df["soft_label"].values.astype(np.float32) * 5000  
    inputs = pos_df[STAT_COLS].fillna(0).values.astype(np.float32)
    
    # 저장된 scaler로 전처리
    scaler = position_scalers[pos]
    inputs_scaled = scaler.transform(inputs)
    X = torch.tensor(inputs_scaled, dtype=torch.float32)
    with torch.no_grad():
        pred = position_models[pos](X).squeeze().numpy()
    
    axs[i].hist(pred, bins=30, alpha=0.7, label="예측값")
    axs[i].hist(targets, bins=30, alpha=0.7, label="실제값")
    axs[i].set_title(f"[{pos}] 실력 점수 분포")
    axs[i].set_xlabel("실력 점수")
    axs[i].set_ylabel("빈도수")
    axs[i].legend()
    axs[i].grid(True)

plt.tight_layout()
plt.savefig("skill_score_distribution_subplot.png")
print("📊 분포 시각화 완료: skill_score_distribution_subplot.png")